# Import Module

In [1]:
import requests
import re
from bs4 import BeautifulSoup
from time import sleep
from random import random

# User Input

In [2]:
def user_input():
    while True:
        try:
            movie_name = input("請輸入欲搜尋的電影名稱：")
            if movie_name.strip():
                return movie_name
        except:
            print('\nTry Again')
            return None

def search_movie(query):
    # search
    encoded_query = requests.utils.quote(query)
    url = f"https://movies.yahoo.com.tw/moviesearch_result.html?keyword={encoded_query}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    movie_titles = soup.find_all("div", class_="release_movie_name")

    # check
    def search_check(movie_titles):
        res_count = len(movie_titles)
        if res_count == 0:
            print(f'沒有找到{query}')
            return None
        elif res_count > 1:
            for m in movie_titles:
                ans = None
                while ans != 'Y' and ans != 'N':
                    ans = input(f'找到多部電影，返回第一個搜尋{m.a.text}(Y)或是往下搜索(N):')
                if ans == 'Y':
                    return m.a.text, m.a['href']
                elif ans == 'N':
                    continue
            # 該次搜尋都沒找到，重新搜尋
            search_movie(user_input())
            sleep(1 + random())
        else:
            return movie_titles[0].a['href']
    
    return search_check(movie_titles)

# Get specific moive infomation

In [3]:
def specific_moive_info(movie_link):
    def review_latest(review_link):
        if review_link == None:
            return None
        res = []
        response = requests.get(review_link)
        soup = BeautifulSoup(response.text, "html.parser")
        reviews = soup.find_all('div', class_='usercom_inner _c')
        for i in reviews[:5]:
            # res.append(i.find_all('span')[-1].text.strip().replace('\r\n', ''))
            res.append(re.sub(r'[\s]{2,}|[\n\r\t]+','',i.find_all('span')[-1].text))
        return res if res else '該電影目前無評論'

    # movie & reviewlink
    movie_info = {}
    response = requests.get(movie_link)
    soup = BeautifulSoup(response.text, "html.parser")
    movie = soup.find('div', class_ = 'movie_intro_info_r')
    review = soup.find('div', class_ ='btn_plus_more usercom_more gabtn')
    review_link = review.a['href'] if review else None
    starscore = soup.find('div', class_ = 'score_num count')
    starbox = soup.find('div', class_ = 'starbox2')

    # moiveinfo
    chname = re.sub(r'[\s]{2,}|[\n\r\t]+','',movie.h1.text)
    engname = re.sub(r'[\s]{2,}|[\n\r\t]+','',movie.h3.text) if movie.h3.text else None
    
    if starscore and starbox:
        starscore = f'{starscore.text} / 5 {starbox.text.strip()}'
    else:
        starscore = None
    
    reviews = review_latest(review_link)

    release_date, IMDB = None, None
    for i in movie.find_all('span'):
        text = i.text.strip()
        if '上映日期' in text:
            release_date = text.split('：')[1]
        elif 'IMDb分數' in text:
            IMDB = f"{text.split('：')[1]} / 10"
    movie_info['中文名稱'] = chname
    movie_info['英文名稱'] = engname
    movie_info['上映日期'] = release_date
    movie_info['滿意度'] = starscore
    movie_info['IMDb'] = IMDB
    movie_info['最新5則評論'] = reviews

    return movie_info

# Ouput

In [10]:
res = search_movie(user_input())
movie_info = specific_moive_info(res[1]) if res else None
movie_info

{'中文名稱': '動物方城市',
 '英文名稱': 'Zootopia',
 '上映日期': '2016-02-26',
 '滿意度': '4.7 / 5 (共1293人投票)',
 'IMDb': '8.1 / 10',
 '最新5則評論': ['現在的動畫即使是大人也能看的很樂在其中，有趣的劇情、還有一些寓意',
  '看似給小朋友看的動畫片，其實是有深度、有寓意，就算是大人也能看的津津樂道的一部好片，毫不猶豫的給5分。',
  '以動物的角色反映現實的社會環境~~~~~~~~~~~~~~~~~~~~~',
  '歌曲很棒兔子敏捷，看似弱小但能力很強樹懶非常令人印象深刻，太好笑太可愛了^^看外表不能代表一切，是要經過彼此的了解及相處的才不會有所謂的刻板印象。',
  '兔子很敏捷 歌很好聽 劇情也很好 幾乎沒有任何缺點不要排擠別人 看似弱小的人其實也是很強的']}